In [13]:
import numpy as np
import torch
import torch.nn as nn

# Training data
The training data can be represented using 2 matrices: inputs and targets, each with one row per observation, and one column per variable.

In [14]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [15]:
# 将numpy格式转换为tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Dataset and DataLoader

In [16]:
from torch.utils.data import TensorDataset

In [17]:
# 定义数据集
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [18]:
from torch.utils.data import DataLoader

In [19]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [20]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.]])
tensor([[103., 119.],
        [ 22.,  37.],
        [119., 133.],
        [ 56.,  70.],
        [ 81., 101.]])


# nn.Linear

In [21]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3535, -0.4681, -0.0698],
        [ 0.1084, -0.2257, -0.3158]], requires_grad=True)
Parameter containing:
tensor([ 0.4155, -0.1487], requires_grad=True)


In [22]:
# 显示模型参数
list(model.parameters())

[Parameter containing:
 tensor([[-0.3535, -0.4681, -0.0698],
         [ 0.1084, -0.2257, -0.3158]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4155, -0.1487], requires_grad=True)]

In [23]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-59.7543, -20.9372],
        [-77.4128, -30.3577],
        [-97.1111, -39.2768],
        [-58.3547, -10.4836],
        [-73.7981, -36.4421]], grad_fn=<ThAddmmBackward>)

# Loss Function

In [24]:
# Import nn.functional
import torch.nn.functional as F

In [25]:
# Define loss function
loss_fn = F.mse_loss

In [26]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(20453.2793, grad_fn=<MseLossBackward>)


tensor([[ 0.0459,  2.1601,  0.1397],
        [ 1.0085, -0.8646,  1.0640]], requires_grad=True)
tensor([[10129.5957, 11335.7783,  6785.2344],
        [-1631.0518, -3661.5190, -1788.0413]])


# Optimizer

In [27]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

# Train the model
1.前向传播
2.计算loss
3.反向传播
4.调整权重
5.梯度归零

In [28]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [29]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 1182.7494
Epoch [20/100], Loss: 557.2047
Epoch [30/100], Loss: 484.7711
Epoch [40/100], Loss: 430.1602
Epoch [50/100], Loss: 382.1826
Epoch [60/100], Loss: 339.8730
Epoch [70/100], Loss: 302.5526
Epoch [80/100], Loss: 269.6271
Epoch [90/100], Loss: 240.5731
Epoch [100/100], Loss: 214.9296


In [30]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 61.3906,  75.3686],
        [ 83.8482,  98.9240],
        [108.1685, 128.7953],
        [ 45.1083,  66.0421],
        [ 90.8378,  99.1327]], grad_fn=<ThAddmmBackward>)

In [31]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [32]:
# 保存模型
torch.save(model.state_dict(), './multiple_linear.pth')